## Preprocessing Pipeline

In [ ]:
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os
import math
from scipy import stats
from scipy.stats import zscore
from mne.preprocessing import peak_finder
from pipeline import CleaningPipe, ICAPipe

Data Structures

In [222]:
control = ['AA4',  'AD5', 'AD7',  'BP5',  'BZ4', 
           'DC9',  'KS6', 'LK4',  'LS8',  'MG1',
           'MK39', 'ML6', 'MU6',  'OS8',  'RA64', 
           'RS2', 'SA50', 'TBN8', 'VB10', 'YL02']
parkinsons = ['AC4',  'AH67', 'AP7', 'AS98', 'ED8',
              'HR67', 'MM4',  'NF1', 'RK4',  'RT1', 
              'SA7',  'SB2',  'TY0', 'VH5',  'VP2',
              'YAA0', 'YG2',  'YG8', 'YH8' , 'YP8',
              'YZ6',  'YZ64', 'ZZ0']

groups = [control, parkinsons]
group_names = ['control', 'parkinsons']

### Data Resampling

In [ ]:
failed = []
success = []
for num, group in enumerate(groups):
    for id in group:
        pipe = CleaningPipe(
            file=rf"D:\eeg\maya\data\subjects\{group_names[num]}\{id}\{id}.mff",
            output_directory=rf"D:\eeg\maya\data\subjects\{group_names[num]}\{id}\outputs",)
        try:
            pipe.resample(sfreq=250,save=True)
            success.append(id)
        except:
            failed.append(id)
print(len(failed), " failed:", failed)
print(len(success)," succeeded:", success)

### Applying Filters

In [ ]:
no_file = []
no_filter = []

for num, group in enumerate(groups):    
    for id in group:
        pipe = 0
        try:
            pipe = CleaningPipe(
                path_to_eeg=rf"D:\eeg\maya\data\subjects\control\{id}\resampled_250\saved_raw\resampled_250hz_raw.fif",
                output_directory=rf"D:\eeg\maya\data\subjects\control\{id}")    
        except:
            no_file.append(id)
            del pipe
            continue
        try:
            pipe.filter(l_freq = 10, h_freq=100, savefig=False)
            pipe.notch()
            pipe._save_raw('EMG_filter.fif')
            success.append(id)
        except:
            no_filter.append(id)

### Extraction of facial Electrodes

In [37]:
subset = ['E94', 'E190', #reference points
          'E226', 'E230', 'E234', 
          'E235', 'E238', 'E239', 'E240', 
          'E241', 'E242', 'E243', 'E244', 
          'E245', 'E248', 'E252']

In [ ]:
fail = []
success = []
for i, group in enumerate(groups):
    for id in group:
        pipe = 0
        try: 
            pipe = CleaningPipe(path_to_eeg=rf"D:\eeg\maya\data\subjects\{group_names[i]}\{id}\saved_raw\EMG_filter.fif")
            pipe.mne_raw.save(rf"D:\eeg\maya\data\EMG\REM_Segments_new\{group_names[i]}\{id}\filtered_subset_raw.fif", picks = subset)
            success.append(id)
        except:
            print(rf"couldn't save {id}'s subset")
            fail.append(id)

### Extraction of REM Segments

In [38]:
def segment_count(group, id):
    count = 0
    dir_path =rf"D:\eeg\maya\data\EMG\REM_Segments\{group}\{id}\REM_segment_{count}_raw.fif"
    while (os.path.exists(dir_path)):
        count+=1
        dir_path =rf"D:\eeg\maya\data\EMG\REM_Segments\{group}\{id}\REM_segment_{count}_raw.fif"
    return count-1

def belongs(stages, mark):
    for i in range(len(stages)):
        if (mark == stages[i]):
            return True
    return False

In [ ]:
stage = [4] # An array that contains the desired stages of sleep
          #0 : WAKE 
          #1 : N1   
          #2 : N2  
          #3 : N3   
          #4 : REM  
fail = []
success = []
for i, group in enumerate(groups):
    for id in group:
        pipe = 0
        try:
            fname = rf"D:\eeg\maya\data\EMG\ICA\REM_Segments\{group_names[i]}\{id}\filtered_subset_raw.fif"
            raw = mne.io.read_raw_fif(fname, verbose = 'ERROR') 
            #print("----- Getting EMG (data) -----")
            data = raw.get_data()
            raw.load_data()
            #print("----- Reading Hypnospectogram (stages) -----")
            stages = np.loadtxt(rf"D:\eeg\maya\data\subjects\{group_names[i]}\{id}\{id}_staging.txt", dtype=int)
            last = stages[-1]
            pad = data.shape[1] - stages.shape[0]
            if pad < 0:
                stages = stages[:data.shape[1]]

                indexes = [i for i in range(len(stages)) if belongs(stage, stages[0][i])]
            else:
                stages.shape = (1,stages.shape[0])
                padding = np.full(pad, last)
                padding.shape = (1,padding.shape[0])
                stages = np.concatenate([stages,padding],axis = 1)
                indexes = [i for i in range(stages.shape[1]) if belongs(stage, stages[0][i])]
            
            #print("----- Extracting REM Ranges -----")
            REM_ranges = []
            start = indexes[0]
            end = indexes[1499]
            for k in range(1500, len(indexes), 1500):
                if k + 1499 < len(indexes):
                    end_index = k + 1499
                else:
                    end_index = len(indexes) - 1
                if indexes[k] - indexes[k-1] == 1:
                    end = indexes[end_index]
                else:
                    REM_ranges.append((start, end))
                    start = indexes[k]
                    end = indexes[end_index]
            REM_ranges.append((start, end))
            #print("----- Saving REM Ranges -----")
            for j,REM_range in enumerate(REM_ranges):
                raw.save(rf"D:\eeg\maya\data\EMG\REM_Segments\{group_names[i]}\{id}\segment_{j}_raw.fif", 
                                tmin = REM_range[0]/250, 
                                tmax = REM_range[1]/250, verbose = 'ERROR')
            success.append(id)

        except Exception as e:
            print(rf"couldn't segment {id}. recieved this error:")
            print(e)
            fail.append(id)

### Channel Interpolation

In [39]:
def segment_count(group, id, stage):
    count = 0
    dir_path =rf"D:\eeg\maya\data\EMG\{stage}_Segments\{group}\{id}\segment_{count}_raw.fif"
    while (os.path.exists(dir_path)):
        count+=1
        dir_path =rf"D:\eeg\maya\data\EMG\{stage}_Segments\{group}\{id}\segment_{count}_raw.fif"
    return count

In [40]:
left_electrodes = ['E241', 'E242', 'E243', 'E244', 'E245', 'E248', 'E252']  #left facial electrodes names
right_electrodes = ['E238', 'E239', 'E240', 'E234', 'E235', 'E230', 'E226'] #right facial electrodes names
left_reference = 'E94'   #electrode names of the left mastoid
right_reference = 'E190' #electrode name of the right mastoid

In [41]:
# A dictionary containing each referenced electrode and its corresponding coordinate location
coordinates = {
    'E226': [7.243, 4.801, -4.772],
    'E230': [6.742, 5.991, -5.833],
    'E234': [5.887, 7.227, -6.547],
    'E235': [6.659, 5.641, -7.657],
    'E238': [4.961, 8.227, -6.783],
    'E239': [5.813, 6.421, -8.650],
    'E240': [6.044, 5.371, -9.814],
    'E241': [-4.961, 8.227, -6.783],
    'E242': [-5.813, 6.421, -8.650],
    'E243': [-6.044, 5.371, -9.814],
    'E244': [-5.887, 7.227, -6.547],
    'E245': [-6.659, 5.641, -7.657],
    'E248': [-6.742, 5.991, -5.833],
    'E252': [-7.243, 4.801, -4.772]
}

In [ ]:
left = [0, 0, 0]
left_distances =[0]*7
right = [0, 0, 0]
right_distances =[0]*7
for i in range(7):
    for j in range(3):
        left[j]+=(1/7)*coordinates[left_electrodes[i]][j]
        right[j]+=(1/7)*coordinates[right_electrodes[i]][j]
for i in range(7):
    left_distances[i] = 1/np.linalg.norm(np.asarray(left)-np.asarray(coordinates[left_electrodes[i]]))
    right_distances[i] = 1/np.linalg.norm(np.asarray(right)-np.asarray(coordinates[right_electrodes[i]]))
left_weights = [i/sum(left_distances) for i in left_distances]
right_weights = [i/sum(right_distances) for i in right_distances]

In [ ]:
stage = 'REM'
for idx, group in enumerate(groups):
    for id in group:
        for segment_num in range(segment_count(group_names[idx],id, stage)):
            fname = rf"D:\eeg\maya\data\EMG\{stage}_Segments\{group_names[idx]}\{id}\segment_{segment_num}_raw.fif"
            raw = mne.io.read_raw_fif(fname, verbose='ERROR')
            raw.load_data(verbose ='ERROR')

            #Fetching Data
            left = raw.copy().pick_channels(left_electrodes, verbose='ERROR')
            left_ref = raw.copy().pick_channels([left_reference], verbose='ERROR')

            right = raw.copy().pick_channels(right_electrodes, verbose='ERROR')
            right_ref = raw.copy().pick_channels([right_reference], verbose ='ERROR')

            left_data = left.get_data(verbose ='ERROR')
            left_ref_data = left_ref.get_data(verbose ='ERROR')
            right_data = right.get_data(verbose ='ERROR')
            right_ref_data = right_ref.get_data(verbose ='ERROR')
            
            #Calculate channels
            size = len(left_data[1])
            left_channel = -1*(left_ref_data)
            right_channel = -1*(right_ref_data)
            for i in range(size):
                for j in range(7):
                    left_channel[0][i] += left_weights[j]*left_data[j][i]
                    right_channel[0][i] += right_weights[j]*right_data[j][i]
            
            # Save channels to .fif file
            ch_names = ['Left', 'Right']
            ch_types = ['emg'] * 2
            info = mne.create_info(ch_names, ch_types=ch_types, sfreq=250)
            new_raw = mne.io.RawArray(np.vstack((left_channel, right_channel)), info, verbose='ERROR')
            new_raw.save(rf"D:\eeg\maya\data\EMG\{stage}_Segments\{group_names[idx]}\{id}\left_right\channels_{segment_num}_raw.fif", verbose = 'ERROR')

### Sampling

In [221]:
x = 10 #sample length (in seconds)
stages = ['REM'] #The array receives the stages the user wants to sample
for stage in stages:    
    for idx, group in enumerate(groups):
        for id in group:
            for segment_num in range(segment_count(group_names[idx],id, stage)): 
                fname = rf"D:\eeg\maya\data\EMG\{stage}_Segments\{group_names[idx]}\{id}\left_right\channels_{segment_num}_raw.fif"
                raw = mne.io.read_raw_fif(fname, verbose='ERROR')
                raw.load_data(verbose='ERROR')
                data = raw.get_data(verbose='ERROR')
                size = data.shape[1]/250
                t = 0
                sample_num = 0
                while (t + x < size):
                    raw.save(rf"D:\eeg\maya\data\EMG\{x}_sec_samples\{stage}\{group_names[idx]}\{id}_{segment_num}_{sample_num}_raw.fif", 
                            tmin = t, 
                            tmax = t + x,
                            verbose='ERROR')
                    t+=x
                    sample_num+=1
            #The commented code below adds the reminder of the segment that is less than a second long as a seperate segment. After some considiration, we decided it would be better to omit these samples. 
                # raw.save(rf"D:\eeg\maya\data\EMG\{x}_sec_samples\{stage}\{group_names[idx]}\{id}_{segment_num}_{sample_num}_raw.fif", 
                #             tmin = t, 
                #             tmax = size,
                #             verbose='ERROR')

### Feature Extraction

In [43]:
def is_zeros(signal):
    return not np.any(signal)

In [44]:
def is_mostly_zeros(signal):
    ''' Check whether most of the signal is a sequence of 0's '''
    size = len(signal)
    max_zeros = 0
    count = 0
    for i in range(size):
        if signal[i] == 0:
            count+=1
            if count > max_zeros:
                max_zeros = count
        else:
            count = 0
    return max_zeros>=size/2

In [45]:
def normalize(sig1, sig2,prop_1,prop_2, func):
    return (func(sig1)*prop_1 + func(sig2)*prop_2)/(prop_1+prop_2)

def normalize_diff(sig1, sig2,prop_1,prop_2, func):
    abs_diff = abs(func(sig1)-func(sig2))
    return abs_diff/normalize(sig1, sig2,prop_1,prop_2, func)

In [46]:
def abs_mean(arr):
    return (np.mean(np.abs(arr)))

def zero_crossing_rate(sig):
    crossings = 0
    l = len(sig)
    for i in range(1, l):
        if (sig[i-1] >= 0 and sig[i] < 0) or (sig[i-1] < 0 and sig[i] >= 0):
            crossings += 1
    return crossings / (l - 1)

In [72]:
def sample_peak_to_peak(sample):
    return max(sample)-min(sample)

In [228]:
def subject_peak_to_peak(path, subject, normalized = True):
    maxi = max_1 = max_2 = float('-inf')
    mini = min_1 = min_2 = float('inf')
    for filename in os.listdir(path):
        #check whether file belongs to subject
        arr = filename.split('_')
        if arr[0]!=subject:
            continue    
        #if so, open file
        f = os.path.join(path, filename)
        raw = mne.io.read_raw_fif(f, verbose='ERROR')
        sig1, sig2 = raw.get_data()
        if is_zeros(sig1) or is_zeros(sig2):
            continue
        if normalized:
            sig1 = zscore(sig1)
            sig2 = zscore(sig2)

        #look for max and min value over all REM samples
        curr_max_1 = max(sig1)
        curr_max_2 = max(sig2)
        
        curr_min_1 = min(sig1)
        curr_min_2 = min(sig2)
        min_1, max_1 = update_min_max(max_1, min_1, curr_max_1, curr_min_1)
        min_2, max_2 = update_min_max(max_2, min_2, curr_max_2, curr_min_2)
        curr_max = max(max_1, max_2)
        curr_min = min(min_1, min_2)
        mini, maxi = update_min_max(maxi, mini, curr_max, curr_min)

    return maxi - mini, max_1 - min_1, max_2 - min_2
        

In [198]:
def update_min_max(maxi, mini, curr_max, curr_min):
    #print(maxi, mini, curr_max, curr_min)
    if maxi < curr_max:
        maxi = curr_max
    if mini > curr_min:
        mini = curr_min
    return mini, maxi

In [225]:
x = 10 #1/5/30 - the sample length used

In [226]:
REM_c_path = rf"D:\eeg\maya\data\EMG\{x}_sec_samples\REM\control"
REM_p_path = rf"D:\eeg\maya\data\EMG\{x}_sec_samples\REM\parkinsons"
REM_t_path = rf"D:\eeg\maya\data\EMG\{x}_sec_samples\REM\test"
paths = [REM_c_path, REM_p_path]
size_c = len(os.listdir(REM_c_path))
size_p = len(os.listdir(REM_p_path))

Feature calculation

In [ ]:
peak_to_peak = [[0 for _ in range(20)], [0 for _ in range(23)]]
peak_to_peak_diff = [[0 for _ in range(20)], [0 for _ in range(23)]]
for num, g in enumerate(groups):
    for i, code in enumerate(g):
        p2p = subject_peak_to_peak(paths[num], code)
        peak_to_peak[num][i] = p2p[0]
        peak_to_peak_diff[num][i] = abs(p2p[1]-p2p[2])

In [230]:
df = pd.DataFrame(columns=['Subject', 'Group', 
          'MAV', 'MAV Diff', 'ZCR', 'ZCR Diff','PAV', 'Peak Count', 'Peak Count Diff',
          'Peak Amp. diff', 'Mean Peak Amp. Diff', 'Mean Peak Delay', 'STD Peak delay', 
          'P2P Diff', 'Sub P2P Diff', 'Sub P2P', 
          'Max Corr', 'Mean Corr', 'mean absolute diff','Norm', 'Pearson'])

for group,path in enumerate([REM_c_path, REM_p_path]):
    for filename in os.listdir(path):
        arr = filename.split('_') 
        
        #extract subject's code and index
        sub_code = arr[0]
        sub_idx = groups[group].index(sub_code)

        #open file
        f = os.path.join(path, filename)
        raw = mne.io.read_raw_fif(f, verbose='ERROR')
        sig1, sig2 = raw.get_data()
        if is_mostly_zeros(sig1) or is_mostly_zeros(sig2):
            continue
        
        #Normalization - zscore on signal level
        sig1 = zscore(sig1)
        sig2 = zscore(sig2)

        #find peaks in signal
        peak_locs_1, peak_mags_1 = peak_finder(sig1, verbose = 'ERROR') 
        peak_locs_2, peak_mags_2 = peak_finder(sig2, verbose = 'ERROR') 

        
        MAV = normalize(sig1, sig2, 1, 1, abs_mean)
        MAV_diff = normalize_diff(sig1, sig2, 1, 1, abs_mean)

        zcr = normalize(sig1, sig2, 1, 1, zero_crossing_rate)
        zcr_diff = normalize_diff(sig1, sig2, 1, 1, zero_crossing_rate)
        #peak related features
        MAVPeak = normalize(peak_mags_1, peak_mags_2, len(peak_locs_1), len(peak_locs_2), abs_mean)
        peak_count = normalize(peak_locs_1,peak_locs_2,1,1,len)
        peak_count_diff = normalize_diff(peak_locs_1,peak_locs_2,1,1,len)
        min_len = min(len(peak_locs_1), len(peak_locs_2))
        amp_diff = np.mean((np.abs(np.subtract(peak_mags_1[:min_len],peak_mags_2[:min_len]))))
        avg_amp_diff = np.abs(np.mean(peak_mags_1[:min_len])- np.mean(peak_mags_2[:min_len]))
        time_diff = np.abs(np.subtract(peak_locs_1[:min_len],peak_locs_2[:min_len]))
        avg_delay = np.mean(time_diff)
        std_delay = np.std(time_diff)
        
        p2p_diff = abs(sample_peak_to_peak(sig1)-sample_peak_to_peak(sig2))
        sub_p2p = peak_to_peak[group][sub_idx]
        sub_p2p_diff = peak_to_peak_diff[group][sub_idx]
        
        coeff = (np.corrcoef(sig1, sig2)[0][1])
        mean_diff = (np.mean(np.abs(sig1 - sig2)))
        corr = np.correlate(sig1, sig2, mode='same')
        max_corr = max(corr)
        mean_corr = np.mean(corr)
        d = np.linalg.norm(sig1 - sig2)        

        df.loc[len(df)] = [sub_code, group, 
                           MAV, MAV_diff, zcr, zcr_diff, MAVPeak, peak_count, peak_count_diff, 
                           amp_diff, avg_amp_diff, avg_delay, std_delay,
                           p2p_diff, sub_p2p_diff, sub_p2p,
                           max_corr, mean_corr, mean_diff, d, coeff]


In [231]:
df.head()

,Subject,Group,MAV,MAV Diff,ZCR,ZCR Diff,PAV,Peak Count,Peak Count Diff,Peak Amp. diff,...,Mean Peak Delay,STD Peak delay,P2P Diff,Sub P2P Diff,Sub P2P,Max Corr,Mean Corr,mean absolute diff,Norm,Pearson
0,AA4,0,0.739529,0.054471,0.1798,0.095662,1.782317,80.5,0.335404,0.726628,...,365.776119,172.998906,0.399074,5.985009,34.436445,1539.706148,0.095131,0.686913,43.847323,0.615636
1,AA4,0,0.759741,0.002793,0.1726,0.081112,1.793669,86.5,0.104046,0.685964,...,138.682927,90.418657,0.249826,5.985009,34.436445,1406.071737,-0.025471,0.732706,46.795903,0.562204
2,AA4,0,0.733916,0.007951,0.1896,0.033755,1.858423,75.0,0.000000,0.868716,...,91.960000,58.242582,0.438770,5.985009,34.436445,1470.152994,0.371051,0.706244,45.405881,0.587826
3,AA4,0,0.722980,0.000657,0.2126,0.084666,1.910027,70.5,0.297872,0.918352,...,398.833333,220.913193,1.636861,5.985009,34.436445,1710.163009,-0.081499,0.631842,39.770265,0.683792
4,AA4,0,0.746014,0.013353,0.1950,0.006154,1.810744,83.5,0.107784,0.595421,...,134.240506,108.649153,0.073166,5.985009,34.436445,1535.396915,0.205252,0.709559,43.945491,0.613913


In [232]:
df.tail()

,Subject,Group,MAV,MAV Diff,ZCR,ZCR Diff,PAV,Peak Count,Peak Count Diff,Peak Amp. diff,...,Mean Peak Delay,STD Peak delay,P2P Diff,Sub P2P Diff,Sub P2P,Max Corr,Mean Corr,mean absolute diff,Norm,Pearson
14450,ZZ0,1,0.762075,0.016777,0.3732,0.055734,1.697270,189.0,0.137566,0.719525,...,101.914773,55.255384,1.248981,12.157826,46.961829,946.725425,0.198635,0.855561,55.754364,0.378539
14451,ZZ0,1,0.786873,0.016812,0.4250,0.036706,1.538453,243.0,0.238683,0.640439,...,184.308411,116.711595,0.749368,12.157826,46.961829,729.333406,-0.002327,0.946792,59.525904,0.291617
14452,ZZ0,1,0.779659,0.006549,0.3936,0.010163,1.664057,189.0,0.137566,0.601009,...,132.278409,93.060200,0.484677,12.157826,46.961829,625.530676,-0.057192,0.962994,61.244907,0.250112
14453,ZZ0,1,0.660086,0.107501,0.3696,0.093074,2.485919,91.0,0.175824,1.076112,...,122.506024,258.807291,1.085649,12.157826,46.961829,262.697743,0.070079,0.881028,69.697199,0.028849
14454,ZZ0,1,0.781087,0.019957,0.4478,0.152747,1.645060,229.5,0.065359,0.632667,...,88.126126,64.086248,0.391919,12.157826,46.961829,382.957809,0.049007,1.023833,65.085209,0.153122


In [218]:
df.to_csv(rf"D:\eeg\maya\data\features_{x}s.csv", index = False)

In [ ]:
#sub_code = group_names[group].index('AA4')
df.columns.tolist()

In [ ]:
'''This method seperates the data into a training set and a test set with x:(1-x) 
proportions such that each subject's samples are either all in the training 
set or all in the test set'''

import itertools

def find_closest_subarrays(set1, set2, x):
    total_set1 = sum(set1)
    total_set2 = sum(set2)
    m = min(total_set1, total_set2)
    target_sum = m * x

    closest_sum_set1 = float('inf')
    closest_subarray_set1 = []

    for r in range(1, len(set1) + 1):
        for indices in itertools.combinations(range(len(set1)), r):
            subgroup = [set1[i] for i in indices]
            subarray_sum = sum(subgroup)
            if abs(subarray_sum - target_sum) < abs(closest_sum_set1 - target_sum):
                closest_sum_set1 = subarray_sum
                closest_subarray_set1 = np.asarray(indices)

    closest_sum_set2 = float('inf')
    closest_subarray_set2 = []

    for r in range(1, len(set2) + 1):
        for indices in itertools.combinations(range(len(set2)), r):
            subgroup = [set2[i] for i in indices]
            subarray_sum = sum(subgroup)
            if abs(subarray_sum - target_sum) < abs(closest_sum_set2 - target_sum):
                closest_sum_set2 = subarray_sum
                closest_subarray_set2 = np.asarray(indices)

    return closest_subarray_set1, closest_subarray_set2